# 1. Chia Train/Test và Augment Dữ Liệu (Back Translation)

## 1.1. Import Thư Viện

In [ ]:
import os
import random
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from googletrans import Translator
import time
import torch

# Set seed
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)


print('Đã import thành công các thư viện!')

✅ Đã import thành công các thư viện!


## 1.2. Cấu Hình Tham Số

In [ ]:
# Đường dẫn
DATA_PATH = './data/dataset.csv'
OUTPUT_DIR = './split_augmented_data'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Tham số chia dữ liệu
TEST_SIZE = 0.2  # 20% cho test, 80% cho train

# Tham số augmentation (CHỈ ÁP DỤNG CHO TRAIN)
AUGMENT_RATIO = 0.5  # Tăng 50% dữ liệu train

# Back translation params
INTERMEDIATE_LANGS = ['de', 'fr']  # Dùng nhiều ngôn ngữ để đa dạng hơn
BACKTRANS_DELAY = 0.5  # Delay giữa các request để tránh rate limit
MAX_BACKTRANS_LENGTH = 500  # Độ dài tối đa của text để back translate

print(f'Input: {DATA_PATH}')
print(f'Output directory: {OUTPUT_DIR}/')
print(f' Test size: {TEST_SIZE} (train size: {1-TEST_SIZE})')
print(f' Augment ratio (train only): {AUGMENT_RATIO}')
print(f' Back Translation - Languages: {INTERMEDIATE_LANGS}, Max length: {MAX_BACKTRANS_LENGTH}')
print(f' Delay between requests: {BACKTRANS_DELAY}s')
print(f'\nCHỈ sử dụng Back Translation để augment dữ liệu!')

📁 Input: ./data/dataset.csv
📁 Output directory: ./split_augmented_data/
⚙️  Test size: 0.2 (train size: 0.8)
⚙️  Augment ratio (train only): 0.5
⚙️  Back Translation - Languages: ['de', 'fr'], Max length: 500
⚙️  Delay between requests: 0.5s

💡 CHỈ sử dụng Back Translation để augment dữ liệu!


## 1.3. Load và Chia Dữ Liệu Gốc

In [ ]:
print('Đang load dữ liệu gốc...')
df = pd.read_csv(DATA_PATH)

print(f'\nĐã load {len(df):,} samples')
print(f'\nPhân phối nhãn gốc:')
print(df['sentiment'].value_counts().sort_index())

# Chia train/test - QUAN TRỌNG: Stratify để giữ cân bằng nhãn
print(f'\nĐang chia dữ liệu train/test ({100*(1-TEST_SIZE):.0f}%/{100*TEST_SIZE:.0f}%)...')
train_df, test_df = train_test_split(
    df,
    test_size=TEST_SIZE,
    random_state=SEED,
    stratify=df['sentiment']  # Giữ cân bằng nhãn
)

print(f'\nĐã chia dữ liệu:')
print(f'   Train: {len(train_df):,} samples')
print(f'   Test: {len(test_df):,} samples')

print(f'\nPhân phối nhãn train:')
print(train_df['sentiment'].value_counts().sort_index())

print(f'\nPhân phối nhãn test:')
print(test_df['sentiment'].value_counts().sort_index())

📂 Đang load dữ liệu gốc...

✅ Đã load 50,000 samples

📊 Phân phối nhãn gốc:
sentiment
negative    25000
positive    25000
Name: count, dtype: int64

🔄 Đang chia dữ liệu train/test (80%/20%)...

✅ Đã chia dữ liệu:
   Train: 40,000 samples
   Test: 10,000 samples

📊 Phân phối nhãn train:
sentiment
negative    20000
positive    20000
Name: count, dtype: int64

📊 Phân phối nhãn test:
sentiment
negative    5000
positive    5000
Name: count, dtype: int64

✅ Đã load 50,000 samples

📊 Phân phối nhãn gốc:
sentiment
negative    25000
positive    25000
Name: count, dtype: int64

🔄 Đang chia dữ liệu train/test (80%/20%)...

✅ Đã chia dữ liệu:
   Train: 40,000 samples
   Test: 10,000 samples

📊 Phân phối nhãn train:
sentiment
negative    20000
positive    20000
Name: count, dtype: int64

📊 Phân phối nhãn test:
sentiment
negative    5000
positive    5000
Name: count, dtype: int64


## 1.4. Lưu Test Set (KHÔNG AUGMENT)

In [ ]:
# Lưu test set nguyên bản - KHÔNG ĐƯỢC AUGMENT
test_path = os.path.join(OUTPUT_DIR, 'test_original.csv')
test_df.to_csv(test_path, index=False)

print(f'Đã lưu test set (KHÔNG augment): {test_path}')
print(f'   Số samples: {len(test_df):,}')
print(f'   💡 Test set này sẽ KHÔNG bao giờ được augment!')

✅ Đã lưu test set (KHÔNG augment): ./split_augmented_data\test_original.csv
   Số samples: 10,000
   💡 Test set này sẽ KHÔNG bao giờ được augment!


## 1.5. Định Nghĩa Back Translator

In [ ]:
class BackTranslator:
    """
    Back translation: English → Intermediate Language → English
    Tạo phiên bản paraphrase giữ nguyên ý nghĩa
    CHỈ SỬ DỤNG CHO TẬP TRAIN!
    """
    
    def __init__(self, intermediate_langs=['de', 'fr', 'es']):
        self.translator = Translator()
        self.intermediate_langs = intermediate_langs
        print(f'BackTranslator initialized')
        print(f'   Intermediate languages: {intermediate_langs}')
    
    def back_translate(self, text, intermediate_lang=None, max_retries=3):
        """
        Back translate text: en → intermediate_lang → en
        
        Args:
            text: Text gốc (English)
            intermediate_lang: Ngôn ngữ trung gian (nếu None thì chọn random)
            max_retries: Số lần retry nếu lỗi
        
        Returns:
            Back-translated text hoặc text gốc nếu lỗi
        """
        if not text or len(text.strip()) == 0:
            return text
        
        # Chọn ngôn ngữ trung gian
        if intermediate_lang is None:
            intermediate_lang = random.choice(self.intermediate_langs)
        
        for attempt in range(max_retries):
            try:
                # Step 1: English → Intermediate Language
                translated = self.translator.translate(
                    text, 
                    src='en', 
                    dest=intermediate_lang
                )
                intermediate_text = translated.text
                
                # Small delay
                time.sleep(0.2)
                
                # Step 2: Intermediate Language → English
                back_translated = self.translator.translate(
                    intermediate_text,
                    src=intermediate_lang,
                    dest='en'
                )
                result = back_translated.text
                
                return result
                
            except Exception as e:
                if attempt < max_retries - 1:
                    # Wait longer before retry
                    time.sleep(2 ** attempt)
                    continue
                else:
                    # Return original text if all retries fail
                    return text
        
        return text

print('Đã định nghĩa BackTranslator class')

✅ Đã định nghĩa BackTranslator class


In [ ]:
back_translator = BackTranslator(intermediate_langs=INTERMEDIATE_LANGS)
print('Sẵn sàng để augment TRAIN SET với Back Translation!')

✅ BackTranslator initialized
   Intermediate languages: ['de', 'fr']
✅ Sẵn sàng để augment TRAIN SET với Back Translation!


## 1.7. Test Augmentation

In [ ]:
# Test với 3 samples từ TRAIN set
print('Testing augmentation với mẫu từ TRAIN set...\n')
test_samples = train_df.sample(3, random_state=42)

for idx, row in test_samples.iterrows():
    original = row['review']
    
    # Test back translation
    truncated = original[:MAX_BACKTRANS_LENGTH] if len(original) > MAX_BACKTRANS_LENGTH else original
    bt_augmented = back_translator.back_translate(truncated)
    
    print(f"{'='*80}")
    print(f"Sentiment: {row['sentiment']}")
    print(f"\nOriginal:")
    print(f"   {original[:150]}..." if len(original) > 150 else f"   {original}")
    print(f"\nBack-translated:")
    print(f"   {bt_augmented[:150]}..." if len(bt_augmented) > 150 else f"   {bt_augmented}")
    print()
    
    time.sleep(BACKTRANS_DELAY)

🧪 Testing augmentation với mẫu từ TRAIN set...

Sentiment: negative

📝 Original:
   Why, oh why, is this trash considered a classic? I've seen higher body counts on episodes of The Simpsons. Virtually nothing happens in this film and ...

🌍 Back-translated:
   Why, oh why, is this garbage considered a classic? I've seen a higher body count in The Simpsons episodes. Virtually nothing happens in this movie and...

Sentiment: negative

📝 Original:
   Why, oh why, is this trash considered a classic? I've seen higher body counts on episodes of The Simpsons. Virtually nothing happens in this film and ...

🌍 Back-translated:
   Why, oh why, is this garbage considered a classic? I've seen a higher body count in The Simpsons episodes. Virtually nothing happens in this movie and...

Sentiment: negative

📝 Original:
   First off, Mexican Werewolf in Texas' title is misleading as many others have pointed out. It is actually about El Chupacabra, which is a similar crea...

🌍 Back-translated:
   Fir

## 1.8. Augment CHỈ Tập TRAIN

In [ ]:
print('='*80)
print('CHUẨN BỊ AUGMENT TẬP TRAIN (BACK TRANSLATION)')
print('='*80)

# Tính số samples cần augment cho mỗi class trong TRAIN set
train_samples_per_class = train_df['sentiment'].value_counts().to_dict()
augment_per_class = {label: int(count * AUGMENT_RATIO) 
                     for label, count in train_samples_per_class.items()}

print(f'\nSố samples cần augment cho TRAIN set:')
total_augment = sum(augment_per_class.values())
for label in sorted(train_df['sentiment'].unique()):
    print(f'   Sentiment {label}: {augment_per_class[label]:,} samples')

# Ước tính thời gian (back translation)
bt_time_est = total_augment * (4 + BACKTRANS_DELAY)  # ~4s per sample + delay
total_time_est = bt_time_est / 60

print(f'\n Thời gian ước tính:')
print(f'   Back Translation: ~{total_time_est:.1f} phút ({total_augment:,} samples)')

# Ước tính thời gian cho MỖI SENTIMENT
print(f'\n Thời gian ước tính CHO MỖI SENTIMENT:')
for label in sorted(train_df['sentiment'].unique()):
    bt_time = augment_per_class[label] * (4 + BACKTRANS_DELAY) / 60
    print(f'   Sentiment {label}: ~{bt_time:.1f} phút')

print(f'\nMỗi sentiment có thể chạy riêng trong các cell khác nhau!')

# Khởi tạo list để lưu augmented data
# QUAN TRỌNG: Chạy cell này TRƯỚC KHI chạy các cell augment bên dưới!
augmented_train_data = []
print(f'\nĐã khởi tạo augmented_train_data = []')

🚀 CHUẨN BỊ AUGMENT TẬP TRAIN (BACK TRANSLATION)

📊 Số samples cần augment cho TRAIN set:
   Sentiment negative: 10,000 samples
   Sentiment positive: 10,000 samples

⏱️  Thời gian ước tính:
   Back Translation: ~1500.0 phút (20,000 samples)

⏱️  Thời gian ước tính CHO MỖI SENTIMENT:
   Sentiment negative: ~750.0 phút
   Sentiment positive: ~750.0 phút

💡 Mỗi sentiment có thể chạy riêng trong các cell khác nhau!

✅ Đã khởi tạo augmented_train_data = []


## 1.8.1. Augment Sentiment "negative"

In [ ]:
sentiment_label = "negative"

print(f'\n{"="*80}')
print(f'Đang augment TRAIN sentiment {sentiment_label}')
print(f'{"="*80}')

# Lấy tất cả samples của class này từ TRAIN
class_samples = train_df[train_df['sentiment'] == sentiment_label]

# ===== BACK TRANSLATION AUGMENTATION =====
num_augment = augment_per_class[sentiment_label]
print(f'\nBack Translation ({num_augment:,} samples)...')

if num_augment > 0:
    bt_samples = class_samples.sample(n=num_augment, replace=True, random_state=SEED)
    
    start_time = time.time()
    success_count = 0
    
    for idx, row in tqdm(bt_samples.iterrows(), total=num_augment, desc=f'BackTrans {sentiment_label}'):
        original = row['review']
        # Truncate để tránh quá dài
        truncated = original[:MAX_BACKTRANS_LENGTH] if len(original) > MAX_BACKTRANS_LENGTH else original
        
        back_translated = back_translator.back_translate(truncated)
        if back_translated and back_translated != truncated:
            augmented_train_data.append({
                'review': back_translated,
                'sentiment': sentiment_label,
                'augment_method': 'BackTranslation'
            })
            success_count += 1
        
        time.sleep(BACKTRANS_DELAY)
    
    bt_elapsed = time.time() - start_time
    print(f'   Hoàn thành BackTrans: {success_count}/{num_augment} thành công trong {bt_elapsed/60:.1f} phút')
else:
    print(f'   Bỏ qua BackTranslation (num_augment = 0)')

print(f'\nĐã augment xong sentiment {sentiment_label}!')
print(f'   Total augmented: {success_count:,} samples')


🔄 Đang augment TRAIN sentiment negative

🌍 Back Translation (10,000 samples)...


BackTrans negative:   0%|          | 0/10000 [00:00<?, ?it/s]

   ✅ Hoàn thành BackTrans: 9997/10000 thành công trong 580.2 phút

✅ Đã augment xong sentiment negative!
   Total augmented: 9,997 samples


## 1.8.2. Augment Sentiment "positive"

In [ ]:
sentiment_label = "positive"

print(f'\n{"="*80}')
print(f'Đang augment TRAIN sentiment {sentiment_label}')
print(f'{"="*80}')

# Lấy tất cả samples của class này từ TRAIN
class_samples = train_df[train_df['sentiment'] == sentiment_label]

# ===== BACK TRANSLATION AUGMENTATION =====
num_augment = augment_per_class[sentiment_label]
print(f'\nBack Translation ({num_augment:,} samples)...')

if num_augment > 0:
    bt_samples = class_samples.sample(n=num_augment, replace=True, random_state=SEED)
    
    start_time = time.time()
    success_count = 0
    
    for idx, row in tqdm(bt_samples.iterrows(), total=num_augment, desc=f'BackTrans {sentiment_label}'):
        original = row['review']
        # Truncate để tránh quá dài
        truncated = original[:MAX_BACKTRANS_LENGTH] if len(original) > MAX_BACKTRANS_LENGTH else original
        
        back_translated = back_translator.back_translate(truncated)
        if back_translated and back_translated != truncated:
            augmented_train_data.append({
                'review': back_translated,
                'sentiment': sentiment_label,
                'augment_method': 'BackTranslation'
            })
            success_count += 1
        
        time.sleep(BACKTRANS_DELAY)
    
    bt_elapsed = time.time() - start_time
    print(f'   Hoàn thành BackTrans: {success_count}/{num_augment} thành công trong {bt_elapsed/60:.1f} phút')
else:
    print(f'   Bỏ qua BackTranslation (num_augment = 0)')

print(f'\nĐã augment xong sentiment {sentiment_label}!')
print(f'   Total augmented: {success_count:,} samples')


🔄 Đang augment TRAIN sentiment positive

🌍 Back Translation (10,000 samples)...


BackTrans positive:   0%|          | 0/10000 [00:00<?, ?it/s]

   ✅ Hoàn thành BackTrans: 9998/10000 thành công trong 579.4 phút

✅ Đã augment xong sentiment positive!
   Total augmented: 9,998 samples


## 1.8.3. Tổng Kết Augmentation

In [ ]:
print(f'\n{"="*80}')
print(f'ĐÃ HOÀN THÀNH TẤT CẢ AUGMENTATION!')
print(f'{"="*80}')

print(f'\nTổng kết:')
print(f'   Tổng augmented samples: {len(augmented_train_data):,}')

# Thống kê theo method
if len(augmented_train_data) > 0:
    df_aug_temp = pd.DataFrame(augmented_train_data)
    
    print(f'\nPhân bố theo phương pháp:')
    for method, count in df_aug_temp['augment_method'].value_counts().items():
        pct = count / len(df_aug_temp) * 100
        print(f'   {method}: {count:,} ({pct:.1f}%)')
    
    print(f'\nPhân bố theo sentiment:')
    for sentiment, count in df_aug_temp['sentiment'].value_counts().sort_index().items():
        pct = count / len(df_aug_temp) * 100
        print(f'   {sentiment}: {count:,} ({pct:.1f}%)')
else:
    print(f'\nChưa có dữ liệu augmented! Hãy chạy các cell 1.8.1 và 1.8.2')


✅ ĐÃ HOÀN THÀNH TẤT CẢ AUGMENTATION!

📊 Tổng kết:
   Tổng augmented samples: 19,995

📊 Phân bố theo phương pháp:
   BackTranslation: 19,995 (100.0%)

📊 Phân bố theo sentiment:
   negative: 9,997 (50.0%)
   positive: 9,998 (50.0%)


## 1.9. Kết Hợp Train Gốc và Augmented

In [ ]:
# Tạo DataFrame augmented train (bỏ cột augment_method trước khi kết hợp)
df_augmented_train = pd.DataFrame(augmented_train_data)
df_augmented_train = df_augmented_train[['review', 'sentiment']]  # Chỉ giữ 2 cột cần thiết

# Kết hợp train gốc với augmented train
train_combined = pd.concat([train_df, df_augmented_train], ignore_index=True)

# Shuffle
train_combined = train_combined.sample(frac=1, random_state=SEED).reset_index(drop=True)

print('='*80)
print('THỐNG KÊ DỮ LIỆU SAU KHI AUGMENT')
print('='*80)

print(f'\nTRAIN SET (ĐÃ AUGMENT - BACK TRANSLATION):')
print(f'   Original train: {len(train_df):,} samples')
print(f'   Augmented (Back Translation): {len(df_augmented_train):,} samples')
print(f'   Total train: {len(train_combined):,} samples')
print(f'\n   Phân phối nhãn train (sau augment):')
for label, count in train_combined['sentiment'].value_counts().sort_index().items():
    pct = count / len(train_combined) * 100
    print(f'      Sentiment {label}: {count:,} ({pct:.1f}%)')

print(f'\nTEST SET (KHÔNG AUGMENT - NGUYÊN BẢN):')
print(f'   Total test: {len(test_df):,} samples')
print(f'\n   Phân phối nhãn test (nguyên bản):')
for label, count in test_df['sentiment'].value_counts().sort_index().items():
    pct = count / len(test_df) * 100
    print(f'      Sentiment {label}: {count:,} ({pct:.1f}%)')

print('\n' + '='*80)

📊 THỐNG KÊ DỮ LIỆU SAU KHI AUGMENT

🔵 TRAIN SET (ĐÃ AUGMENT - BACK TRANSLATION):
   Original train: 40,000 samples
   Augmented (Back Translation): 19,995 samples
   Total train: 59,995 samples

   Phân phối nhãn train (sau augment):
      Sentiment negative: 29,997 (50.0%)
      Sentiment positive: 29,998 (50.0%)

🔴 TEST SET (KHÔNG AUGMENT - NGUYÊN BẢN):
   Total test: 10,000 samples

   Phân phối nhãn test (nguyên bản):
      Sentiment negative: 5,000 (50.0%)
      Sentiment positive: 5,000 (50.0%)



## 1.10. Lưu Dữ Liệu

In [ ]:
print('Đang lưu dữ liệu...\n')

# Lưu train augmented
train_path = os.path.join(OUTPUT_DIR, 'train_augmented.csv')
train_combined.to_csv(train_path, index=False)
print(f'Đã lưu TRAIN (augmented): {train_path}')
print(f'   Samples: {len(train_combined):,}')
print(f'   Size: {os.path.getsize(train_path) / 1024 / 1024:.2f} MB')

# Test đã được lưu ở bước 1.4
print(f'\nTEST (original - đã lưu trước đó): {test_path}')
print(f'   Samples: {len(test_df):,}')
print(f'   Size: {os.path.getsize(test_path) / 1024 / 1024:.2f} MB')

💾 Đang lưu dữ liệu...

✅ Đã lưu TRAIN (augmented): ./split_augmented_data\train_augmented.csv
   Samples: 59,995
   Size: 59.66 MB

✅ TEST (original - đã lưu trước đó): ./split_augmented_data\test_original.csv
   Samples: 10,000
   Size: 12.60 MB
✅ Đã lưu TRAIN (augmented): ./split_augmented_data\train_augmented.csv
   Samples: 59,995
   Size: 59.66 MB

✅ TEST (original - đã lưu trước đó): ./split_augmented_data\test_original.csv
   Samples: 10,000
   Size: 12.60 MB


## 1.11. Tổng Kết

In [ ]:
print('\n' + '='*80)
print('HOÀN THÀNH CHIA VÀ AUGMENT DỮ LIỆU!')
print('='*80)

print(f'\nTổng Kết:')
print(f'   Input: {DATA_PATH}')
print(f'   Original dataset: {len(df):,} samples')
print(f'   Train/Test split: {100*(1-TEST_SIZE):.0f}%/{100*TEST_SIZE:.0f}%')
print(f'   Files đã tạo:')
print(f'      1. {train_path}')
print(f'         - Train gốc: {len(train_df):,} samples')
print(f'         - Augmented (Back Translation): {len(df_augmented_train):,} samples')
print(f'         - Tổng: {len(train_combined):,} samples')
print(f'\n      2. {test_path}')
print(f'         - Test gốc (KHÔNG augment): {len(test_df):,} samples')

print('\n' + '='*80)

# Giải phóng bộ nhớ
del back_translator
print('\n🧹 Đã giải phóng bộ nhớ')


🎉 HOÀN THÀNH CHIA VÀ AUGMENT DỮ LIỆU!

📊 Tổng Kết:
   ✅ Input: ./data/dataset.csv
   ✅ Original dataset: 50,000 samples
   ✅ Train/Test split: 80%/20%
   📁 Files đã tạo:
      1. ./split_augmented_data\train_augmented.csv
         - Train gốc: 40,000 samples
         - Augmented (Back Translation): 19,995 samples
         - Tổng: 59,995 samples

      2. ./split_augmented_data\test_original.csv
         - Test gốc (KHÔNG augment): 10,000 samples

⚠️  LƯU Ý QUAN TRỌNG:
   ✅ Tập TRAIN đã được augment bằng Back Translation:
      • Back Translation: Paraphrase qua ngôn ngữ trung gian
   ✅ Tập TEST giữ NGUYÊN - KHÔNG augment
   ✅ Tránh data leakage - đánh giá khách quan!

💡 Bước tiếp theo:
   1. Chạy notebook 2_encode_split_data.ipynb để mã hóa dữ liệu
   2. Chạy notebook 3_train_with_split.ipynb để train model


🧹 Đã giải phóng bộ nhớ
